In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd


## Load the trained model ,scaler pickle, OHE

### load model

In [39]:
model = load_model('model.h5')

### load the encdoer and scaler

In [40]:
with open('Oht_Geography.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)


with open('Label_Encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('sclaer.pkl','rb') as file:
    scaler = pickle.load(file)

### Example of input data


In [42]:
input_data= {
    'CreditScore':600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
    
    }

#### Apply One-Hot-encoder 

In [43]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))

d:\ANN-project\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [44]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [45]:
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [46]:
# cpnvert data from dictionry into DtaFrame
input_data_df = pd.DataFrame([input_data])
input_data_df 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


#### Label Encoder

In [47]:
input_data_df['Gender']=label_encoder_gender.fit_transform(input_data_df['Gender'])

In [48]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000


#### Concat new features from OHE and atuel data

In [49]:
input_df = pd.concat([input_data_df.drop(["Geography"],axis=1),geo_encoded_df],axis=1)

In [50]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


#### Scaling Transform Data

In [51]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

## Prediction

In [52]:
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 1s 597ms/step


array([[0.03058057]], dtype=float32)

In [55]:
prediction_proba = prediction[0][0]
prediction_proba

0.03058057

In [57]:
if prediction_proba > 0.5:
    print("the customer is likely to churn")
else:
    print("the customer is not likely to churn")
    

the customer is not likely to churn
